In [65]:
import os
os.environ['TF_KERAS'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import onnx
import keras
import onnx2keras
from onnx2keras import onnx_to_keras


import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

import keras2onnx

from tensorflow.python.keras.models import load_model
from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread

import cv2
from tqdm import tqdm
import random
import pickle


import torchvision.transforms as transforms
from torchvision.utils import save_image
from torchvision.transforms import functional
from PIL import Image

print("tf.__version__ is", tf.__version__)
print("tf.keras.__version__ is:", tf.keras.__version__)

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


PIL_transform = transforms.ToPILImage()
tensor_transform = transforms.ToTensor()



tf.__version__ is 2.2.0
tf.keras.__version__ is: 2.3.0-tf
Num GPUs Available:  1


In [67]:
k_model.get_layer(index=0).kernel_regularizer = regularizers.l2(0.0001)
k_model.get_layer(index=0).bias_regularizer = regularizers.l2(0.0001)

k_model.get_layer(index=1).kernel_regularizer = regularizers.l2(0.0001)
k_model.get_layer(index=1).bias_regularizer = regularizers.l2(0.0001)

k_model.get_layer(index=2).kernel_regularizer = regularizers.l2(0.0001)
k_model.get_layer(index=2).bias_regularizer = regularizers.l2(0.0001)

k_model.get_layer(index=3).kernel_regularizer = regularizers.l2(0.0001)
k_model.get_layer(index=3).bias_regularizer = regularizers.l2(0.0001)

k_model.get_layer(index=4).kernel_regularizer = regularizers.l2(0.0001)
k_model.get_layer(index=4).bias_regularizer = regularizers.l2(0.0001)

k_model.get_layer(index=5).kernel_regularizer = regularizers.l2(0.0001)
k_model.get_layer(index=5).bias_regularizer = regularizers.l2(0.0001)

k_model.get_layer(index=6).kernel_regularizer = regularizers.l2(0.0001)
k_model.get_layer(index=6).bias_regularizer = regularizers.l2(0.0001)


X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1,1,IMG_SIZE,IMG_SIZE, 1)
y = np.array(y)

pickle_out = open("X.pickle", "wb")
pickle.dump(X,pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y,pickle_out)
pickle_out.close()

In [68]:
X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

#X = X/255.0
VAL_PCT = 0.1  # lets reserve 10% of our data for validation
val_size = int(len(X)*VAL_PCT)

train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]


In [69]:
k_model.compile(loss="mean_squared_error", optimizer="adam",metrics=['accuracy'])
k_model.fit(train_X,train_y, batch_size=1, epochs=100)

Epoch 1/100
4569/4569 [==============================] - 14s 3ms/step - loss: 0.0548 - accuracy: 0.9315
Epoch 2/100
4569/4569 [==============================] - 13s 3ms/step - loss: 0.0232 - accuracy: 0.9709
Epoch 3/100
4569/4569 [==============================] - 13s 3ms/step - loss: 0.0147 - accuracy: 0.9812
Epoch 4/100
4569/4569 [==============================] - 13s 3ms/step - loss: 0.0107 - accuracy: 0.9873
Epoch 5/100
4569/4569 [==============================] - 14s 3ms/step - loss: 0.0084 - accuracy: 0.9895
Epoch 6/100
4569/4569 [==============================] - 13s 3ms/step - loss: 0.0064 - accuracy: 0.9926
Epoch 7/100
4569/4569 [==============================] - 14s 3ms/step - loss: 0.0078 - accuracy: 0.9917
Epoch 8/100
4569/4569 [==============================] - 14s 3ms/step - loss: 0.0065 - accuracy: 0.9919
Epoch 9/100
4569/4569 [==============================] - 14s 3ms/step - loss: 0.0052 - accuracy: 0.9932
Epoch 10/100
4569/4569 [==============================] - 14s 3m

In [70]:
k_model.evaluate(test_X, test_y, batch_size=1)

507/507 [==============================] - 1s 2ms/step - loss: 0.0187 - accuracy: 0.9803


[0.01865518093109131, 0.9802761077880859]

In [66]:
onnx_model = onnx.load("Material_Classifier.onnx")
k_model = onnx_to_keras(onnx_model, ['imageinput'])

keras.models.save_model(k_model, "Material_Classifier.h5", overwrite=True,include_optimizer=True)


IMG_SIZE=200
CARBON = "CARBON/"
FIBERGLASS = "FIBERGLASS/"
LABELS = {CARBON: 0, FIBERGLASS: 1}
training_data = []

carbon_count = 0
fiberglass_count = 0


for label in LABELS:
    for f in tqdm(os.listdir(label)):
        if "jpg" in f:
            try:
                path = os.path.join(label,f)
                img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
                
                #---------------------------------------------
                '''
                plt.imshow(img, cmap = "gray")
                plt.show()
                '''
                #---------------------------------------------
                
                
                # randomize numbers ----------------------------------------------------------------------------------
                #degrees = abs(random.uniform(0.0,10.0)) # Get the degree range
                #scale = random.uniform(.5,2.0) # Get the scale range
                #horizontal_translation_factor = random.uniform(0.0,.2) # Get the translation factor range
                #vertical_translation_factor = random.uniform(0.0,.2)
                # randomize numbers ----------------------------------------------------------------------------------            
                # transform assuming that it does the operation implicitly and needs no variable assigning
                #PIL_image = PIL_transform(img)
                #PIL_image = transforms.functional.affine(PIL_image,degrees,(horizontal_translation_factor, vertical_translation_factor),scale,shear = 0, resample=0,fillcolor=None)
                # if label == self.NEGATIVE:
                    # save = PIL_image.save("Transforms/Bad/" + f)

                # elif label == self.POSITIVE:
                    # save = PIL_image.save("Transforms/Good/" + f)
                
                #tensor_transform(PIL_image)

                training_data.append([np.array(img), np.eye(2)[LABELS[label]]])



                if label == CARBON:

                    carbon_count =carbon_count + 1
                elif label == FIBERGLASS:

                    fiberglass_count = fiberglass_count + 1
            except Exception as e:
                print("bad things")
                pass

    np.random.shuffle(training_data)
    print('carbon:', carbon_count)
    print('fiberglass:', fiberglass_count)

#training_data = np.load("training_data.npy", allow_pickle=True)
print("training data length:", len(training_data))

INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:None
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> imageinput_Mean.
DEBUG:onnx2keras:Input 1 -> conv_1_W.
DEBUG:onnx2keras:Input 2 -> conv_1_B.
DEBUG:onnx2keras:Input 3 -> batchnorm_1_scale.
DEBUG:onnx2keras:Input 4 -> batchnorm_1_B.
DEBUG:onnx2keras:Input 5 -> batchnorm_1_mean.
DEBUG:onnx2keras:Input 6 -> batchnorm_1_var.
DEBUG:onnx2keras:Input 7 -> conv_2_W.
DEBUG:onnx2keras:Input 8 -> conv_2_B.
DEBUG:onnx2keras:Input 9 -> batchnorm_2_scale.
DEBUG:onnx2keras:Input 10 -> batchnorm_2_B.
DEBUG:onnx2keras:Input 11 -> batchnorm_2_mean.
DEBUG:onnx2keras:Input 12 -> batchnorm_2_var.
DEBUG:onnx2keras:Input 13 -> conv_3_W.
DEBUG:onnx2keras:Input 14 -> conv_3_B.
DEBUG:onnx2keras:Input 15 -> batchnorm_3_scale.
DEBUG:onnx2keras:Input 16 -> batchnorm_3_B.
DEBUG:onnx2keras:Input 17 -> batchnorm_3_mean.
DEBUG:onnx2keras:Input 18 -> batchnorm_3_var.
DEBUG:onnx2keras:Input 19 -> conv_4_W.
DEBUG:onnx2

carbon: 2570
fiberglass: 0


100%|████████████████████████████████████████████████████████████████████████████| 2506/2506 [00:01<00:00, 1396.72it/s]

carbon: 2570
fiberglass: 2506
training data length: 5076
